# Trees and Graphs

In [2]:
class Graph:
    def __init__(self):
        self.nodes = []
    def __str__(self):
        return "\n".join([f"{node}: {[friend.name for friend in node.friends]}" for node in self.nodes])
        # return "".join([f"{f}: {f.friends}" for f in self.nodes])

class Node:
    def __init__(self, name):
        self.friends = []
        self.name = name
    def __str__(self):
        return self.name

s = Node("s")
p = Node("p")
m = Node("m")
l = Node("l")
r = Node("r")
e = Node("e")

s.friends = [p, m]
m.friends = [l]
r.friends = [e]
g = Graph()
g.nodes = [s, p, m, l, r, e]
print(g)

s: ['p', 'm']
p: []
m: ['l']
l: []
r: ['e']
e: []


## Route between nodes

In [3]:
from collections import deque

def isThereARoute(n1, n2):
    # do BFS from n1 to find n2
    q = deque()
    marked = set()
    q.append(n1)
    while q:
        n = q.popleft()
        if n == n2:
            return True
        else:
            if n in marked:
                pass
            else:
                marked.add(n)
                for friend in n.friends:
                    q.append(friend)
    return False
print(isThereARoute(s, e))

False


## Generate Balanced Binary Search Tree from Array

In [4]:
class TreeNode:
    def __init__(self, val):
        self.val = val
        self.left = None
        self.right = None
    def __str__(self):
        return f"{self.val}: ( {self.left} - {self.right} )"
    def __repr__(self):
        return f"{self.val}: ( {self.left} - {self.right} )"

In [5]:
# basically do binary search where each midpoint becomes a node in the tree
a = [1,2,3,4,5,6,7,8,9,10]

def createBST(arr):
    # print(arr)
    if arr:
        l = 0
        r = len(arr) - 1
        mid = (l + r) // 2
        root = TreeNode(arr[mid])
        root.left = createBST(arr[0:mid])
        root.right = createBST(arr[mid+1:])
        return root
    else:
        return None

t = createBST(a)
print(t)

5: ( 2: ( 1: ( None - None ) - 3: ( None - 4: ( None - None ) ) ) - 8: ( 6: ( None - 7: ( None - None ) ) - 9: ( None - 10: ( None - None ) ) ) )


## List of depths

In [6]:
from collections import deque

class ListNode:
    def __init__(self, val):
        self.next = None
        self.val = val
    def __repr__(self):
        return f"{str(self.val)} -> {self.next}"
    
# input binary tree - output a list of singly linked lists where each list contains all the nodes at a certain level of the tree
def listOfDepths(tree):
    # maintain a queue for each depth
    q = deque()
    q.append(tree)
    slls = []

    while q:

        l = len(q)
        current_level_head = None
        current_level_tail = None

        for i in range(l):
            n = q.popleft()
            listnode = ListNode(n.val)

            if not current_level_head:
                current_level_head = listnode
                current_level_tail = listnode
            else:
                current_level_tail.next = listnode
                current_level_tail = current_level_tail.next

            if n.left:
                q.append(n.left)
            if n.right:
                q.append(n.right)
        
        if current_level_head:
            slls.append(current_level_head)
    
    return slls


s = listOfDepths(t)
print(len(s))
print(s)

4
[5 -> None, 2 -> 8 -> None, 1 -> 3 -> 6 -> 9 -> None, 4 -> 7 -> 10 -> None]


## Check balanced

In [7]:
# check if the heights of the left and right subtree of each node differ by more than one
def checkBalanced(tree):
    if not tree:
        return True
    l = getHeight(tree.left)
    r = getHeight(tree.right)
    if abs(l - r) > 1:
        return False
    else:
        return checkBalanced(tree.left) and checkBalanced(tree.right)

def getHeight(tree):
    if not tree:
        return 0
    if (not tree.left) and (not tree.right):
        return 1
    else:
        return 1 + max(getHeight(tree.left), getHeight(tree.right))

t.left.right.right.right = None
print(checkBalanced(t))
t.left.right.right.right = TreeNode(4.5)
print(checkBalanced(t))

True
False


## Validate BST

In [8]:
# check if a binary tree is a BST
def validateBST(root, minn, maxx):
    # for each node check if node.left < node < node.right and that node.left > minmin and node.right < maxmax
    # print(root, minn, maxx)
    if not root:
        return True
    else:
        if (not root.left) and (not root.right):
            # print('returning true')
            return True
        else:
            l = True
            r = True
            if root.left:
                if root.left.val >= root.val or root.left.val < minn:
                    return False
                else:
                    l = validateBST(root.left, minn, root.val)
            if root.right:
                if root.right.val <= root.val or root.right.val > maxx:
                    return False
                else:
                    r = validateBST(root.right, root.val, maxx)
            return r and l

t.left.right.right.right.val = 4.5
print(validateBST(t, -99999, 999999))
t.left.right.right.right.val = 12
print(validateBST(t, -99999, 999999))

True
False


## Successor

In [9]:
# given a bst with backlinks, find the next greater stored value
def successor(node):
    # this will be the leftmost value of the right subtree
    if node.right:
        p = node.right
        while p.left:
            if p.left:
                p = p.left
        return p

    # or if there is no right subtree, work back up the tree until encountering somthing that is greater
    else:
        v = node.val
        p = node
        while p and p.val <= v:
            p = p.parent
        return p

## Build order

In [10]:
class GraphNode:
    def __init__(self, val):
        self.val = val
        self.required_for = set()
    def __repr__(self):
        targets = ', '.join(str(n.val) for n in self.required_for)
        return f"{self.val}: pointing to {targets} |"

In [11]:
# which order to build projects based on dependencies
def buildOrder(projects, dependencies):
    # make DAG
    nodes = {}
    for p in projects:
        nodes[p] = 0
    for required, dependent in dependencies:
        required.required_for.add(dependent)
        nodes[dependent] += 1
    
    # print(nodes)
    # add all projects that have no dependencies to the queue
    q = deque()
    for n, score in nodes.items():
        if score == 0:
            q.append(n)
    
    # if no building possible
    if len(q) == 0:
        return None
    
    # print(q)
    ret = []
    # pop from queue, decrement the dependency count of projects it is required for, and add any that are at zero to the queue
    while q:
        n = q.popleft()
        ret.append(n)
        for p in n.required_for:
            nodes[p] -= 1
            if nodes[p] == 0:
                q.append(p)
    return ret



a = GraphNode('a')
b = GraphNode('b')
c = GraphNode('c')
d = GraphNode('d')
e = GraphNode('e')
f = GraphNode('f')
buildOrder([a,b,c,d,e,f], [(a,d),(f,b),(b,d),(f,a),(d,c)])


[e: pointing to  |,
 f: pointing to b, a |,
 b: pointing to d |,
 a: pointing to d |,
 d: pointing to c |,
 c: pointing to  |]

## First common ancestor

In [12]:
# given a binary tree and two nodes in that tree, return the first common ancestor of the two nodes in the tree

# check if r contains n
def contains(r, n):
    if r == n:
        return True
    elif r is None:
        return False
    else:
        return contains(r.left, n) or contains(r.right, n)

def isLCA(root, n1, n2):
    # if left subtree has it
    if contains(root.left, n1):
        if contains(root.right, n2):
            return root
        else:
            return isLCA(root.left, n1, n2)
        
    if contains(root.left, n2):
        if contains(root.right, n1):
            return root
        else:
            return isLCA(root.left, n1, n2)
    
    if contains(root.right, n1):
        if contains(root.left, n2):
            return root
        else:
            return isLCA(root.right, n1, n2)
        
    if contains(root.left, n2):
        if contains(root.right, n1):
            return root
        else:
            return isLCA(root.right, n1, n2)

one = TreeNode(1)
two = TreeNode(2)
three = TreeNode(3)
four = TreeNode(4)
five = TreeNode(5)
seven = TreeNode(7)
twelve = TreeNode(12)
nine = TreeNode(9)

one.left = two
one.right = three
two.left = four
two.right = five
three.right = seven
four.left = twelve
five.right = nine

isLCA(one, twelve, five)

2: ( 4: ( 12: ( None - None ) - None ) - 5: ( None - 9: ( None - None ) ) )

## BST Sequences

In [ ]:
# first element must always be the root
# can make the left and right subtree from any order
def getAllArrays(root):
    if not root:
        return []
    else:
        if root.left and root.right:
            return getOrderedPerumuations(getAllArrays(root.left), getAllArrays(root.right))
        elif root.left:
            return getOrderedPerumtations(getAllArrays(root.left))
        elif root.right:
            return getOrderedPerumtations(getAllArrays(root.right))
        else:
            return [root.val]

# if [1,2,4] and [5,6,7] are the two arrays,
# ordered perms returns all the permuatations of the union where order of each individual list is maintained, so like [5,6,1,7,2,4]
def getOrderedPermutations(*args):
        if len(args) == 1:
            return 
        
        for el in l:
            el.append(root.val)
        if root.left and root.right:
            x = getAllArrays(root.left, l)
            y = getAllArrays(root.right, l)
            ret = x + y
            return ret
        else:
            if root.left:
                for el in l:
                    el.append(root.left.val)
                
            if root.right:
                for el in l:
                    el.append(root.left.val)
            return l
    else:

getAllArrays(root)
